In [ ]:
%pip install unsloth datasets trl

In [ ]:
from unsloth import FastLanguageModel
from datasets import load_dataset, Dataset
import pandas as pd
from trl import SFTTrainer, SFTConfig
import torch

### Prompt

In [62]:
system_prompt = """
คุณคือ ดร.อเล็กซานเดอร์ โอเลอร์ นักคณิตศาสตร์ระดับโลกและครูสอนคณิตศาสตร์ระดับมัธยมปลาย คุณสร้างโจทย์คณิตศาสตร์ตาม Bloom's Taxonomy

## Bloom's Taxonomy (ไทย)
1. **จำ** - จำและเรียกคืนสูตร ข้อมูลพื้นฐาน
2. **เข้าใจ** - อธิบายความหมาย แนวคิดด้วยคำตนเอง  
3. **นำไปใช้** - ใช้สูตรแก้ปัญหาสถานการณ์ใหม่
4. **วิเคราะห์** - แยกแยะองค์ประกอบ วิเคราะห์ความสัมพันธ์
5. **ประเมิน** - ตัดสิน ประเมินค่า ให้เหตุผลตามเกณฑ์
6. **สร้างสรรค์** - สร้างสิ่งใหม่ ออกแบบ วางแผน

## ขั้นตอนการทำงาน
1. ใน `<think></think>` ให้: คิดโจทย์เป็นอังกฤษก่อน → ทดลองแก้ → ตรวจสอบความถูกต้อง → สร้างตัวเลือก → ตรวจ Bloom's level
2. เขียนผลลัพธ์ในรูปแบบ XML

## รูปแบบ Output
```xml
<questions>
  <question>
    <text>โจทย์ด้วย KaTeX เช่น $ 2x + 3 = 7 $</text>
    <type>multiple_choice หรือ short_answer</type>
    <options>
      <option>$ ตัวเลือก1 $</option>
      <option>$ ตัวเลือก2 $</option>
    </options>
    <correct_answer>$ คำตอบ $</correct_answer>
    <explanation>ขั้นตอนการแก้ด้วย KaTeX</explanation>
    <score>1-5</score>
    <difficulty>ง่าย/ปานกลาง/ยาก</difficulty>
    <bloom_levels>
      <level>เข้าใจ</level>
      <level>นำไปใช้</level>
    </bloom_levels>
  </question>
</questions>
```

## หมายเหตุ
- ใช้ KaTeX `$ ... $` สำหรับคณิตศาสตร์ทั้งหมด
- ระดับความยาก: ง่าย (1-2 คะแนน), ปานกลาง (3-4 คะแนน), ยาก (5+ คะแนน)
- สามารถใช้หลาย Bloom's level ต่อโจทย์
"""

### Cleaning Up the Dataset

In [8]:
dataset = load_dataset("UpMath/problems")

Generating train split: 100%|██████████| 1107/1107 [00:00<00:00, 11769.33 examples/s]


In [ ]:
dataset = dataset["train"]

In [32]:
df = pd.DataFrame(dataset)
df

,topic,grade,question_type,difficulty,bloom,num_problems,user_prompt,success,error,content,reasoning
0,พีชคณิต,ม.4,multiple_choice,ง่าย,[จำ],1,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,```xml\n<questions>\n <question>\n <text>ส...,"\nOkay, let's tackle this request. The user wa..."
1,พีชคณิต,ม.4,multiple_choice,ง่าย,[จำ],2,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>จงหาค่า $...,"\nOkay, let's tackle this request. The user wa..."
2,พีชคณิต,ม.4,multiple_choice,ง่าย,[จำ],3,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>จงหาค่า $...,"\nOkay, I need to create 3 easy multiple-choic..."
3,พีชคณิต,ม.4,multiple_choice,ง่าย,[เข้าใจ],1,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>จงพิจารณา...,"\nOkay, I need to create a high-quality math p..."
4,พีชคณิต,ม.4,multiple_choice,ง่าย,[เข้าใจ],2,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>สมการใดเท...,"\nOkay, let's see. I need to create two multip..."
...,...,...,...,...,...,...,...,...,...,...,...
1102,พีชคณิต,ม.6,multiple_choice,ยาก,"[นำไปใช้, ประเมิน, สร้างสรรค์]",2,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>ออกแบบฟัง...,"\nAlright, I need to create two high-quality m..."
1103,พีชคณิต,ม.6,multiple_choice,ยาก,"[นำไปใช้, ประเมิน, สร้างสรรค์]",3,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>ออกแบบพหุ...,"\nOkay, let's tackle this request. I need to c..."
1104,พีชคณิต,ม.6,multiple_choice,ยาก,"[วิเคราะห์, ประเมิน, สร้างสรรค์]",1,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>นักเรียนค...,"\nOkay, let's tackle this request. The user wa..."
1105,พีชคณิต,ม.6,multiple_choice,ยาก,"[วิเคราะห์, ประเมิน, สร้างสรรค์]",2,จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1....,True,,<questions>\n <question>\n <text>นักเรียนค...,"\nOkay, I need to create two high-quality math..."


In [33]:
# Inspect dataset schema and a few samples
import json, re, math
from typing import List, Dict, Any, Optional
from pathlib import Path

print("Dataset type:", type(dataset))
try:
    # HuggingFace Datasets usually expose .features
    print("Features:", getattr(dataset, "features", None))
except Exception as e:
    print("Could not read features:", e)

print("Columns:", list(dataset.column_names) if hasattr(dataset, "column_names") else [])

# Show first 2 raw records for reference (truncate long strings)
preview = []
for i, rec in enumerate(dataset):
    if i >= 2:
        break
    pr = {}
    for k, v in rec.items():
        sv = str(v)
        pr[k] = (sv[:200] + "…") if len(sv) > 200 else sv
    preview.append(pr)
print(json.dumps(preview, ensure_ascii=False, indent=2))

Dataset type: <class 'datasets.arrow_dataset.Dataset'>
Features: {'topic': Value('string'), 'grade': Value('string'), 'question_type': Value('string'), 'difficulty': Value('string'), 'bloom': List(Value('string')), 'num_problems': Value('int64'), 'user_prompt': Value('string'), 'success': Value('bool'), 'error': Value('string'), 'content': Value('string'), 'reasoning': Value('string')}
Columns: ['topic', 'grade', 'question_type', 'difficulty', 'bloom', 'num_problems', 'user_prompt', 'success', 'error', 'content', 'reasoning']
[
  {
    "topic": "พีชคณิต",
    "grade": "ม.4",
    "question_type": "multiple_choice",
    "difficulty": "ง่าย",
    "bloom": "['จำ']",
    "num_problems": "1",
    "user_prompt": "จงสร้างโจทย์คณิตศาสตร์คุณภาพสูงโดยกำหนดให้\n1. หัวข้อ: พีชคณิต\n2. สำหรับนักเรียน: ม.4\n3. รูปแบบ: multiple_choice\n4. ความยาก: ง่าย\n5. bloom level: จำ\n6. จำนวน: 1 ข้อ\n7. เพิ่มเติม: โจทย์จำเป็นต้องมีคำตอบ…",
    "success": "True",
    "error": "",
    "content": "```xml\n<question

In [34]:
# Transform to example schema
# Target columns as per screenshot: question, choices, correct_answer, correct_option_index, level, subject, explanation, score, difficulty, bloom_level

from dataclasses import dataclass

target_cols = [
    "question",
    "choices",
    "correct_answer",
    "correct_option_index",
    "level",
    "subject",
    "explanation",
    "score",
    "difficulty",
    "bloom_level",
]

# Heuristics to extract from potential fields
# We assume dataset records may contain fields like: text/question, options/choices, answer/correct_answer, explanation, level/grade, subject, difficulty, bloom or bloom_levels

def pick(d: dict, keys: list[str]) -> Optional[Any]:
    for k in keys:
        if k in d and d[k] is not None:
            return d[k]
    return None


def coerce_choices(val) -> list[str]:
    # Accept list[str], list[dict], str (comma/semicolon separated), or XML-like from examples.
    if val is None:
        return []
    if isinstance(val, list):
        # if list of dict with key 'text' or 'option'
        out = []
        for it in val:
            if isinstance(it, dict):
                out.append(str(pick(it, ["text", "option", "value", "label"])) or "")
            else:
                out.append(str(it))
        return [s for s in out if s != ""]
    s = str(val)
    # try to parse simple JSON list
    try:
        maybe = json.loads(s)
        if isinstance(maybe, list):
            return [str(x) for x in maybe]
    except Exception:
        pass
    # split by ; or |
    parts = re.split(r"\s*[,;|]\s*", s)
    return [p for p in parts if p]


def extract_from_xml(xml_text: str) -> tuple[Optional[str], list[str], Optional[str], Optional[str]]:
    # Very light-weight parse for the common structure in your JSON examples
    # Returns: question_text, options, correct_answer, explanation
    if not xml_text:
        return None, [], None, None
    q = None
    expl = None
    # question text
    m = re.search(r"<text>([\s\S]*?)</text>", xml_text)
    if m:
        q = m.group(1).strip()
    # options
    opts = re.findall(r"<option>([\s\S]*?)</option>", xml_text)
    opts = [o.strip() for o in opts]
    # correct answer
    m = re.search(r"<correct_answer>([\s\S]*?)</correct_answer>", xml_text)
    ans = m.group(1).strip() if m else None
    # explanation
    m = re.search(r"<explanation>([\s\S]*?)</explanation>", xml_text)
    if m:
        expl = m.group(1).strip()
    return q, opts, ans, expl


records = []
for rec in dataset:
    # Try common fields
    question = pick(rec, ["question", "text", "prompt", "input"]) or ""
    explanation = pick(rec, ["explanation", "rationale", "solution", "reasoning"]) or ""
    subject = pick(rec, ["subject", "topic"]) or "คณิตศาสตร์"
    level = pick(rec, ["level", "grade"]) or "ม.4"
    difficulty = pick(rec, ["difficulty"]) or "easy"
    bloom = pick(rec, ["bloom", "bloom_level", "bloom_levels"]) or "จำ"
    score = pick(rec, ["score"]) or 2

    # choices and correct
    choices = pick(rec, ["choices", "options"]) or []
    correct_answer = pick(rec, ["correct_answer", "answer"]) or None

    # If there is an xml-like block, pull from it
    xml_blob = pick(rec, ["content"]) if isinstance(pick(rec, ["content"]) , str) else None
    if isinstance(xml_blob, str) and ("<questions>" in xml_blob or "<question>" in xml_blob):
        q2, ch2, ans2, ex2 = extract_from_xml(xml_blob)
        question = q2 or question
        if ch2:
            choices = ch2
        correct_answer = ans2 or correct_answer
        explanation = ex2 or explanation

    # Normalize choices
    choices = coerce_choices(choices)

    # If still missing choices but we have multiple-choice-like fields, attempt to construct from A/B/C/D
    if not choices:
        cands = []
        for key in ["A", "B", "C", "D", "E"]:
            if key in rec and rec[key]:
                cands.append(str(rec[key]))
        choices = cands

    # If correct answer is an index, map to value; if it's a label like 'A', map to index 0
    correct_option_index = None
    if isinstance(correct_answer, (int, float)) and not isinstance(correct_answer, bool):
        idx = int(correct_answer)
        if 0 <= idx < len(choices):
            correct_option_index = idx
            correct_answer = choices[idx]
    else:
        # try match by exact string
        if correct_answer is not None and choices:
            try:
                correct_option_index = choices.index(correct_answer)
            except ValueError:
                # try strip $ and spaces (LaTeX-like)
                norm = correct_answer.strip().strip("$")
                for i, c in enumerate(choices):
                    if norm == str(c).strip().strip("$"):
                        correct_option_index = i
                        correct_answer = c
                        break

    # If still None, try to infer from provided 'correct_option_index'/'answer_index' keys
    if correct_option_index is None:
        idx = pick(rec, ["correct_option_index", "answer_index", "label", "target"])
        try:
            if idx is not None:
                idx = int(idx)
                if idx >= 1 and idx <= len(choices):
                    # some datasets are 1-based
                    correct_option_index = idx - 1
                elif 0 <= idx < len(choices):
                    correct_option_index = idx
                if correct_option_index is not None and (correct_answer is None or correct_answer == ""):
                    correct_answer = choices[correct_option_index]
        except Exception:
            pass

    # Fallbacks
    if correct_answer is None and choices:
        # leave empty but index might remain None
        correct_answer = ""

    bloom_level = None
    if isinstance(bloom, list):
        bloom_level = ", ".join(map(str, bloom))
    else:
        bloom_level = str(bloom)

    row = {
        "question": str(question),
        "choices": choices,
        "correct_answer": str(correct_answer) if correct_answer is not None else "",
        "correct_option_index": int(correct_option_index) if correct_option_index is not None else None,
        "level": str(level),
        "subject": str(subject),
        "explanation": str(explanation),
        "score": int(score) if isinstance(score, (int, float)) else 2,
        "difficulty": str(difficulty),
        "bloom_level": bloom_level,
    }
    records.append(row)

split_df = pd.DataFrame.from_records(records, columns=target_cols)
# Show a small sample
split_df.head(10)

,question,choices,correct_answer,correct_option_index,level,subject,explanation,score,difficulty,bloom_level
0,สมการเชิงเส้นตัวแปรเดียวมีรูปแบบทั่วไปคือข้อใด,"[$ ax^2 + bx + c = 0 $, $ ax + b = 0 $, $ a^2x...",$ ax + b = 0 $,1.0,ม.4,พีชคณิต,สมการเชิงเส้นตัวแปรเดียวมีนิยามว่าเป็นสมการที่...,2,ง่าย,จำ
1,จงหาค่า $ x $ จากสมการ $ 2x + 5 = 13 $,"[$ x = 4 $, $ x = 9 $, $ x = 3 $, $ x = -4 $, ...",$ x = 4 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: ลบ 5 ออกจากทั้งสองข้างของสมการ $...,2,ง่าย,จำ
2,จงหาค่า $ x $ ที่ทำให้สมการ $ x + 3 = 7 $ เป็น...,"[$ 4 $, $ 10 $, $ -4 $, $ 21 $, $ 5 $, $ 8 $, ...",$ 4 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: แยกตัวแปร $ x $ โดยการลบ $ 3 $ อ...,2,ง่าย,จำ
3,จงพิจารณาขั้นตอนการแก้สมการ $ 3x + 2 = 11 $ ข้...,"[$ 3x = 11 - 2 $ ดังนั้น $ 3x = 9 $, $ 3x = 11...",$ 3x = 11 - 2 $ ดังนั้น $ 3x = 9 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: ต้องลบ 2 ออกจากทั้งสองข้างของสมก...,2,ง่าย,เข้าใจ
4,สมการใดเทียบเท่ากับ $ 3(x - 4) = 15 $ หลังจากใ...,"[$ 3x - 12 = 15 $, $ 3x - 4 = 15 $, $ 3x + 12 ...",$ 3x - 12 = 15 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: ใช้คุณสมบัติการแจกแจง $ a(b + c)...,2,ง่าย,เข้าใจ
5,เมื่อแก้สมการ $ 3x - 4 = 11 $ คำตอบที่ถูกต้องค...,"[$ x = 5 $, $ x = \frac{7}{3} $, $ x = \frac{1...",$ x = 5 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: เพิ่ม 4 ทั้งสองข้างของสมการ $ 3x...,2,ง่าย,เข้าใจ
6,จงหาค่า $ x $ จากสมการ $ 3x + 5 = 2x + 10 $,"[$ x = 5 $, $ x = 15 $, $ x = 1 $, $ x = -5 $]",$ x = 5 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: ย้ายตัวแปร $ x $ ไปข้างเดียวกัน ...,2,ง่าย,นำไปใช้
7,จงหาค่า $ x $ จากสมการ $ 3x - 4 = 8 $,"[$ x = 4 $, $ x = 12 $, $ x = \frac{4}{3} $, $...",$ x = 4 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: บวก 4 ทั้งสองข้างของสมการ $ 3x -...,2,ง่าย,นำไปใช้
8,จงหาค่า $ x $ จากสมการ $ 3x + 5 = 14 $,"[$ x = 3 $, $ x = \frac{8}{3} $, $ x = \frac{1...",$ x = 3 $,0.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: ย้ายค่าคงที่ไปอีกฝั่ง $ 3x = 14 ...,2,ง่าย,นำไปใช้
9,ในการแก้สมการ $ 4x + 7 = 2x + 15 $ ขั้นตอนการค...,"[$ 4x - 2x = 15 + 7 $, $ 4x + 2x = 15 - 7 $, $...",$ 4x - 2x = 15 - 7 $,2.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: ย้าย $ 2x $ ไปทางซ้ายและ $ 7 $ ไ...,2,ง่าย,วิเคราะห์


In [35]:
# Validate and save outputs
from pathlib import Path
out_dir = Path("data/processed")
out_dir.mkdir(parents=True, exist_ok=True)

# Basic validation: ensure choices non-empty for multiple-choice
problems = split_df[(split_df["choices"].map(len) == 0)]
print(f"Rows without choices: {len(problems)}")

# Save to CSV and JSONL
csv_path = out_dir / "problems_transformed.csv"
jsonl_path = out_dir / "problems_transformed.jsonl"

split_df.to_csv(csv_path, index=False)
print("Saved:", csv_path)

with open(jsonl_path, "w", encoding="utf-8") as f:
    for _, row in split_df.iterrows():
        rec = row.to_dict()
        # ensure choices are serializable as list
        if isinstance(rec.get("choices"), pd.Series):
            rec["choices"] = rec["choices"].tolist()
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
print("Saved:", jsonl_path)

split_df.sample(min(len(split_df), 5))

Rows without choices: 2
Saved: data/processed/problems_transformed.csv
Saved: data/processed/problems_transformed.jsonl


,question,choices,correct_answer,correct_option_index,level,subject,explanation,score,difficulty,bloom_level
582,นักเรียนต้องการสร้างฟังก์ชันกำลังสองที่มีจุดยอ...,"[$ f(x) = 0.5(x - 1)^2 - 2 $, $ f(x) = 2(x - 1...",$ f(x) = (x - 1)^2 - 2 $,3.0,ม.5,พีชคณิต,ขั้นตอนที่ 1: จำรูปแบบฟังก์ชันกำลังสองในรูปจุด...,2,ปานกลาง,"จำ, ประเมิน, สร้างสรรค์"
280,สมการกำลังสอง $ 2x^2 + kx + 3 = 0 $ มีคำตอบจริ...,"[$ k = 24 $, $ k = 12 $, $ k = \sqrt{24} $, $ ...",$ k = \pm 2\sqrt{6} $,3.0,ม.4,พีชคณิต,ขั้นตอนที่ 1: สมการกำลังสอง $ ax^2 + bx + c = ...,2,ยาก,"เข้าใจ, นำไปใช้"
578,จงหาคำตอบของสมการ $ 2x^2 + 3x - 2 = 0 $ โดยใช้...,"[$ x = \frac{-3 \pm \sqrt{7}}{4} $, $ x = \fra...",$ x = \frac{-3 \pm 5}{4} $,3.0,ม.5,พีชคณิต,"ขั้นตอนที่ 1: ระบุค่า $ a = 2 $, $ b = 3 $, $ ...",2,ปานกลาง,"จำ, วิเคราะห์, ประเมิน"
963,"นักเรียนคนหนึ่งกล่าวว่า ""สมการกำลังสอง $ 3x^2 ...",[ถูกต้อง เนื่องจาก $ \Delta = 36 - 12k $ และเม...,ถูกต้อง เนื่องจาก $ \Delta = 36 - 12k $ และเมื...,0.0,ม.6,พีชคณิต,ขั้นตอนที่ 1: คำนวณค่าดิสคริมิแนนท์ $ \Delta =...,2,ปานกลาง,"เข้าใจ, วิเคราะห์, ประเมิน"
1105,นักเรียนคนหนึ่งอ้างว่าสมการกำลังสอง $ x^2 + (k...,"[$ k &lt; -1 $ หรือ $ k &gt; 7 $, $ -1 &lt; k ...",$ k &lt; -1 $ หรือ $ k &gt; 7 $,0.0,ม.6,พีชคณิต,ขั้นตอนที่ 1: สมการกำลังสองมีรากจริง 2 รากที่แ...,2,ยาก,"วิเคราะห์, ประเมิน, สร้างสรรค์"


In [58]:
dataset_split = Dataset.from_pandas(split_df)
dataset_split.push_to_hub("UpMath/Synthetic")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.57s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/UpMath/Synthetic/commit/1fd22d35242547776e0c85b7659716f04580aec3', commit_message='Upload dataset', commit_description='', oid='1fd22d35242547776e0c85b7659716f04580aec3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/UpMath/Synthetic', endpoint='https://huggingface.co', repo_type='dataset', repo_id='UpMath/Synthetic'), pr_revision=None, pr_num=None)

### Fine-tune

In [ ]:
dataset = load_dataset("UpMath/problems", split="train")

Generating train split: 100%|██████████| 1107/1107 [00:00<00:00, 6575.35 examples/s]


#### prepare the dataset

In [ ]:
def create_full_output(examples):
    content = examples["content"]
    reasoning = examples["reasoning"]

    examples["full_output"] = f"""<think>
{reasoning}
</think>{content}"""
    
    return examples

In [ ]:
dataset = dataset.map(create_full_output, remove_columns=["content", "reasoning"])

#### prepare the model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-4B-Thinking-2507",
    max_seq_length = 19200,
    load_in_4bit = False,
    load_in_8bit = True,
    full_finetuning = False,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

In [ ]:
def generate_conversation(examples):
    user_prompts = examples["user_prompt"]
    assistant_outputs = examples["full_output"]

    conversations = []
    for user_prompt, assistant_output in zip(user_prompts, assistant_outputs):
        conversations.append([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": assistant_output},
        ])

    return {"conversations": conversations}

In [ ]:
reasoning_conversations = tokenizer.apply_chat_template(
    dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize = False,
)

In [ ]:
reasoning_conversations[0]

In [ ]:
sft_config = SFTConfig(
    hub_model_id="UpMath/Thai-HomeworkGen-v4-Lora",
    dataset_text_field="conversations",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    auto_find_batch_size=True,
    num_train_epochs=10,
    warmup_ratio=0.05,
    learning_rate=4e-5,
    optim="paged_adamw_8bit",
    bf16=True,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42069,
    push_to_hub=True,
    report_to="none",
    save_total_limit=2,
    metric_for_best_model="loss",
    greater_is_better=False,
    load_best_model_at_end=True,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=sft_config
    train_dataset=dataset,
    tokenizer=tokenizer,
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.push_to_hub_merged("UpMath/Thai-HomeworkGen-v4", tokenizer, save_method="merged_16bit")